## Import Libraries

In [1]:
from neo4j import GraphDatabase
from pyvis.network import Network
import os
import webbrowser

## Data Loading

In [2]:
# Load config from file
def load_neo4j_config(filepath):
    config = {}
    with open(filepath, 'r') as file:
        for line in file:
            if '=' in line:
                key, value = line.strip().split('=', 1)
                config[key] = value
    return config

In [3]:
# Load credentials from the file
config = load_neo4j_config("../Playground/Encryption/Stack Overflow Sandbox Credentials.txt")

In [4]:
# Assign values
uri = config.get("NEO4J_URI")
username = config.get("NEO4J_USERNAME")
password = config.get("NEO4J_PASSWORD")
database = config.get("NEO4J_DATABASE", "neo4j")

## Initialise Data Driver

In [5]:
# Create driver
driver = GraphDatabase.driver(uri, auth=(username, password))

In [6]:
# Example query to test
def test_connection():
    with driver.session() as session:
        result = session.run("RETURN 'Connection successful!' AS message")
        print(result.single()["message"])

In [7]:
test_connection()

Connection successful!


## Example 1

In [8]:
# Function to run your Cypher query with a parameter
def get_top_answerers_by_tag(tag_name):
    query = """
    MATCH (t:Tag {name:$tagName})<-[:TAGGED]-(q:Question)<-[:ANSWERED]-(a:Answer {is_accepted:true})<-[:PROVIDED]-(u:User)
    RETURN DISTINCT u.display_name as answerer
    LIMIT 5
    """
    with driver.session(database=database) as session:
        result = session.run(query, tagName=tag_name)
        return [record["answerer"] for record in result]

In [9]:
# Example usage
top_answerers = get_top_answerers_by_tag("python")
print("Top Python Answerers:")
for name in top_answerers:
    print("-", name)

Top Python Answerers:
- T. de Jong
- Abhishek Tyagi
- cybersam
- jose_bacoy
- Thomas J


## Example 2

In [10]:
def run_graph_query(tag_name):
    query = """
    MATCH (t:Tag {name:$tagName})<-[:TAGGED]-(q:Question)<-[:ANSWERED]-(a:Answer {is_accepted:true})<-[:PROVIDED]-(u:User)
    RETURN t, q, a, u
    LIMIT 25
    """
    with driver.session(database=database) as session:
        result = session.run(query, tagName=tag_name)
        return list(result)  # 👈 CONVERT TO LIST HERE

In [11]:
def visualise_graph(result):
    import os
    import webbrowser
    from pyvis.network import Network

    os.makedirs("output", exist_ok=True)
    net = Network(notebook=False, height="750px", width="100%", directed=True)
    nodes = set()

    for record in result:
        t = record.get("t")
        q = record.get("q")
        a = record.get("a")
        u = record.get("u")

        for node in [t, q, a, u]:
            if node and node['id'] not in nodes:
                label = node['label'].capitalize()
                name = node['properties'].get('name') or node['properties'].get('title') or node['properties'].get('display_name') or label
                net.add_node(node['id'], label=name, title=label)
                nodes.add(node['id'])

        if t and q:
            net.add_edge(t['id'], q['id'], label="TAGGED")
        if q and a:
            net.add_edge(q['id'], a['id'], label="ANSWERED")
        if a and u:
            net.add_edge(a['id'], u['id'], label="PROVIDED")

    net.write_html("output/neo4j_graph_example_2.html")
    webbrowser.open("file://" + os.path.abspath("output/neo4j_graph_example_2.html"))

In [12]:
def truncate_question_titles(result):
    """
    Rebuilds the query result as a list of mutable dicts,
    with the 'q' (Question) node's title shortened to 8 characters.
    """
    new_result = []

    for record in result:
        new_record = {}

        for key in ['t', 'q', 'a', 'u']:
            node = record.get(key)
            if node:
                props = dict(node.items())  # copy all properties
                if (key == 'q' or key == 'a') and 'title' in props:
                    props['title'] = props['title'][:8] + '...' if len(props['title']) > 8 else props['title']
                new_record[key] = {
                    'id': node.id,
                    'label': key,
                    'properties': props
                }
        new_result.append(new_record)

    return new_result

In [13]:
# Run and visualise
raw_result = run_graph_query("neo4j")
processed_result = truncate_question_titles(raw_result)
visualise_graph(processed_result)

C:\Users\HaochengLin\AppData\Local\Temp\ipykernel_1152\116795794.py:18: DeprecationWarning: `id` is deprecated, use `element_id` instead
  'id': node.id,
